# Introduction

PyWake is an open source wind farm simulation tool capable of calculating wind farm flow fields, power production and annual energy production (AEP) of wind farms.

PyWake provides a unified interface to wind farm models of different fidelities, e.g. different engineering models and CFD-RANS (commercial plugin).

PyWake is implemented in Python, but still very fast due to heavy vectorization and use of numerical libraries.




## The PyWake Philosophy

"Empowering users" underlines the formulation of PyWake. Its highly modular architecture (shown in the figures below) allows users to combine different AEP modelling blocks in all sorts of fashions - giving the flexibility to shape PyWake most optimally around the particularities of real-world problems. Yet with power also comes responsibility - the user needs to make an informed decision when combining the multitude of building blocks PyWake supplies. Essentially everyone can build their *own* AEP model chain leveraging PyWake's flexibility, so there is not *one* PyWake solution either and users should ensure to report the building blocks they used from PyWake to transparently communicate their results.    

## Overview
![PyWake](../_static/WF_Site_WindTurbines.svg)

The `WindFarmModel` is initialized with a `Site` and a `WindTurbines` object.

*Site*: For a given position, reference wind speed (WS<sub>ref</sub>) and wind direction (WD<sub>ref</sub>), `Site` provides the local wind condition in terms of wind speed (WS), wind direction (WD), turbulence intensity (TI) and the probability of each combination of wind direction and wind speed. Furthermore, `Site` is responsible for calculating the down-wind, cross-wind and vertical distance between wind turbines (which in non-flat terrain is different from the straight-line distances).

*WindTurbines*: For a given wind turbine type and effective wind speed (WS<sub>eff</sub>), the `WindTurbines` object provides the power and thrust coefficient (CT), as well as the wind turbine hub height (H) and diameter (D).

*WindFarmModel*: For a given set of wind turbines, defined in terms of their position and type, the `WindFarmModel` returns a `SimulationResult` object containing the calculated the effective wind speed, power production, thrust coefficient as well as methods to calculate the AEP and generate flow maps

## WindFarmModels 

### Engineering models

![Engineering models](../_static/EngineeringModels.svg)

#### Overview

The engineering wind farms models in PyWake are composed of one of two wind farms models in combination with a wake deficit model, a superposition model and optionally a blockage deficit model, turbulence model and rotor average model:

- `EngineeringWindFarmModel`: Defines the proceedure for solving the wind farm simulation
    - `PropagateDownwind`: wakes only
    - `All2AllIterative`: wakes + blockage
- Wake `DeficitModel`: Calculates wake deficit from one wind turbine to downstream wind turbines or sites in the wind farm
    - `groundModel`: Option for modelling the ground effect
    - `use_effective_ws`: Option for using the local wind speed at the rotor to compute the deficit 
    - `use_effective_ti`: Option for using the local turbulence intensity at the rotor to compute TI dependant qunatities
- `SuperpositionModel`: Defines how deficits from multiple sources are combined
- Blockage `DeficitModel`: Calculates blockage deficit (speed-ups) from one wind turbine to upstream/downstream wind turbines or sites in the wind farm
    - `groundModel`: Option for modelling the ground effect
    - `superpositionModel`: Option to define how to combine blockage different from wakes 
- `DeflectionModel`: Defines the wake deflection due to yaw misalignment, sheared inflow etc, by modifying the down- and cross wind distances
- `TurbulenceModel`: Calculates added turbulence in the wake from one wind turbine to downstream wind turbines or sites in the wind farm
- `rotorAvgModel`: Determines the method for computing the effective wind speed at the wind turbine

This list should only serve as a rough overview of the different components constituting the `EngineeringWindFarmModel` and the available options - please refer to the more detailed description of each component and their respective methods before simulating. 

A set of predefined engineering wake models are available in PyWake with more to follow in the near-future, see [Predefined Windfarm models](EngineeringWindFarmModels.ipynb#Predefined-WindFarmModels):

- NOJ
- Fuga
- FugaBlockage (wake and blockage)
- BastankhahGaussian
- IEA37SimpleBastankhahGaussian

##### General modelling considerations

The never ending stream of publications concerning wind farm flows nicely demonstrates the difficulty we are still facing in accurately modelling the aerodynamic phenomena within wind energy. There is currently not a single accepted method that could possibly capture the extensive range of flow scales - from aerofoil boundary layer to farm-to-farm - especially not one that is sufficiently cheap to allow computing AEP. Engineering wind farm models present a reasonable balance between accuracy and costs for wind farm AEP simulations, yet they rely on multiple sub-models that need to be selected wisely according to the problem at hand. A certain model and parameter combination might work for one particular case but not another - as demonstrated in an endless number of published model comparisons/validations. PyWake's engineering wind farm models are therefore also completely modular, allowing to combine different single wake models with different superposition, blockage, turbulence and deflection models to simulate wind farm flows. Each respective model is also configurable and its parameters tunable.

The sheer amount of possible sub-model combinations and settings can be overwhelming, so here some very general guidance for how to combine them and what choices to consider. Let's start with the selection of superposition model - here you mainly have to decide between `SquaredSum` or `LinearSum`. The former does not really have a clear physical foundation, but was found to give some promising results in deep-arrays, as it isolates the most severe deficits, much like `MaxSum`. Beware, though, that its successful application largely depends on how the deficits are calculated inside the farm. Depending on the control volume one considers, one could either use the wind farm free-stream wind speed even for turbines inside the array or if isolating individual turbines, the incoming wind speed at each turbine, which then includes the deficits from upstream turbines (and blockage from upstream/donwstream, if included). Whilst physically it is hard to argue for using the free-stream velocity inside the array - upstream turbines have taken energy out of the flow - one could choose to do so, however then it is imperative not to use `LinearSum`. PyWake uses the *effective wind speed* at each turbine to calculate the thrust, power and deficit, thus if the free-stream velocity is chosen as *effective wind speed*, then the deficits are extremely large even inside the array and the flow goes negative when simply summing them (`LinearSum`). A physically more consistent approach - in-line with using single wake models - is to use the incoming wind speed to the turbine as *effective wind speed* by setting `use_effective_ws=True`. The deficits should then be combined with the `LinearSum` - the `SquaredSum` actually leads to double counting (see Park2 model description for details). The `WeightedSum` approach by Zong et al is only available for Gaussian wake deficit models as of now and tends towards the `LinearSum` results. It is an iterative method with high memory requierements and only differs from the `LinearSum` when combining extremely deep deficits. It should individually be judged whether its benefits outweigh the additional costs. 

Another important selection is the wake deficit model. Here the general recommendation is to use a Gaussian type model. They are in fact faster than the top-hat model, momentum deficit conserving (in isolation) and avoid singular behaviour (step changes). The available Gaussian model versions differ in their formulation of their streamwise wake expansion, which is governed by the initial wake diameter (at the wake origin) and its rate of expansion with downstream distance. The expansion can be constant (pre-set by user) or based on the *effective turbulence intensity (TI)*. Increasing TI leads to greater mixing and thus quicker expansion/deficit attenuation (users can change the coefficients if desired). When using a turbulence model the wake added turbulence in a farm is combined and leads to spatially varying TI. As with the deficit, the user can choose to use the local TI at the rotor (`use_effective_ti=True`) or the free-stream TI. Whilst physically one might argue for using the local TI, this approach leads to excessive wake expansion deep inside the array when summing all contributions. The wake addded turbulence is overestimated and the free-stream TI seems the better reference to set the wake expansion. The reason lies in the simplistic representation of turbulence by a single figure, TI, and its connection to wake expansion. The free-stream TI describes the atmospheric turbulence and thus also the larger scales contributing to wake-meandering and wake breakdown. The wake-added turbulence is small scale and dissipates quicker, thus contributing differently to wake mixing. For this reason in Fuga and the DWM model only the free-stream TI governs wake expansion. One should also keep in mind that the Guassian wake shape is a time-averaged solution, that incorporates meandering, shear-layer breakdown and mixing without differentiating between them. 
Classically Gaussian models are far-wake models, so the near-wake is rather crude. Shapiro et al. introduced a smooth growth in the deficit, which also forms part of the Zong model. The latter also sets the initial wake diameter by the near-wake length to capture the effect of TI and tip speed ratio on near-wake breakdown and subsequent far-wake onset. Other near-wake updates exist, yet their value is limited in AEP computations as turbines are usually placed outside this region in wind farms.

In PyWake it is possible to introduce a mirror plane to represent the ground effect - turbines are mirrored in the ground plane. This is a method commonly employed in vortex models, as it enforces zero wall-normal velocity. However, whilst for blockage models highly recommended, it is less obvious whether to use a mirror plane for wakes. Wake models are generally tuned to measurement or simulation data that intrinsically contain the ground effect - inroducing it, could lead to double counting. This is also the reason why the `groundModel` can be set seperately for blockage and wake models. 

Finally let's discuss the wind turbine model itself. PyWake uses power and CT curves, which can also be multi-dimensional (more suited to wind farm simulations), but necessarily need to be a function of the *free-stream* (reference) wind speed. In PyWake a turbine's blockage and wake effects on itself, as well as its own mirror contributions (performance curves already include the ground effect), are zero. Thus the wind speed at the rotor can be used to interpolate power and CT from the reference curves. The velocity at the rotor can be computed in different ways (`rotorAvgModel`), default is extracting the velocity at the rotor-centre, however one can also opt for more rotor-equivalent wind speed quantities. Yet, one should consider that a turbine's performance curves are usually generated by using the rotor-centre velocity. The wind speed used for interpolation and deficit computation are always identical. As the power and CT curves are non-linear users are advised to use higher-order splines in their interpolation (for instance `pchip`). A final issue relates to the often singular behaviour of power/thrust curves around cut-in and cut-out. Firstly, around those wind-speeds sudden jumps in production can occur as turbines are close to switch on/off, so even small changes in wind speed/direction can lead to strong relative changes. Secondly, in combination with blockage there is no definite solution, as some turbines might be on or off - either would be correct. This is mostly related to the downstream speed-up blockage can cause in addition to upstream deficit. In the `All2AllIterative` on/off oscillations are avoided by tracking unstable turbines and switching them all off. 

We hope the above recommendations will help you getting started with PyWake and enjoy exploring its flexibility in your own work. We are happy to recieve any comments, suggestions and ideas for collaboration. 

The PyWake Team


## EllipSys3D (RANS-CFD) 

The EllipSys wind farm model is based on a Reynolds-Averaged Navier-Stokes method as implemented in the general purpose flow solver EllipSys3D, initially developed by Jess A. Michelsen at DTU Fluid Mechanics[1,2] and Niels N. Sørensen in connection with his PhD study[3]. 

EllipSys3D is closed source licensed software and it is based on Fortran and MPI. The EllipSys wind farm model uses PyEllipSys, which is a direct memory Python interface to the Fortran version of EllipSys3D. 

This means that it is possible to import EllipSys3D as a Python object and change flow variables during a simulation. 

The wind turbines are represented by actuator diks (AD) and the inflow is an idealized atmospheric boundary layer including effects of Coriolis and atmospheric stability. The main setup uses flat terrain, with a homogeneous roughness length, but the EllipSys wind farm model can also be run with terrain. More information can be found here: https://topfarm.pages.windenergy.dtu.dk/cuttingedge/pywake/pywake_ellipsys/


### Installation

The EllipSys wind farm model only runs in a linux environment and it requires the
commercial cutting-edge plugin `py_wake_ellipsys`. Contact us if you are interested to get access to it.


### References

1. Jess A. Michelsen, *Basis3D - a Platform for Development of Multiblock
   PDE Solvers*, AFM 92-05, Department of Fluid Mechanics, Technical University
   of Denmark, December 1994.
2. Jess A. Michelsen, Block structured Multigrid solution of 2D and 3D
   Elliptic PDEs, AFM 94-06, Department of Fluid Mechanics, Technical
   University of Denmark, May 1994.
3. Niels N. Sørensen, *General Purpose Flow Solver Applied to Flow Over Hills*,
   Risø-R-827, Risø National Laboratory, Roskilde, Denmark, 1995.
